In [2]:
import gradio as gr
from transformers import pipeline
from dotenv import load_dotenv
import os
import psycopg2

load_dotenv()  


True

In [4]:

conn = psycopg2.connect(
    host=os.getenv("DB_HOST", "localhost"),
    database=os.getenv("DB_NAME"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD")
)

In [5]:
translation_pipeline = pipeline("translation_en_to_de", model="Helsinki-NLP/opus-mt-en-de")
results = translation_pipeline("I love you")
results[0]['translation_text']

Device set to use cpu


'Ich liebe dich'

In [10]:
def translate_transformers(from_text):
    results = translation_pipeline(from_text)
    translated_text = results[0]['translation_text']

    with conn.cursor() as cursor:
        cursor.execute(
            "INSERT INTO translation_history (original_text, translated_text) VALUES (%s, %s)",
            (from_text, translated_text)
        )
    conn.commit()

    return translated_text


In [11]:
translate_transformers("I love Aziret")

'Ich liebe Aziret'

In [12]:
interface = gr.Interface(fn=translate_transformers, inputs=gr.Textbox(lines=2, placeholder="Text to translate"), 
                         outputs="text")

In [13]:
interface.launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
